In [4]:
!pip install fastBPE sacremoses

In [5]:
import torch
# English to German translation
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
en2de.cuda()
en2de.translate("Machine learning is great!")  # 'Maschinelles Lernen ist großartig!'




Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


'Maschinelles Lernen ist großartig!'

In [6]:
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')
de2en.cuda()
de2en.translate("Maschinelles Lernen ist großartig!")

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 2992273886/2992273886 [02:05<00:00, 23766221.25B/s]


'Machine learning is great!'

In [11]:
import time
start = time.time()
out = de2en.translate(en2de.translate("I am testing a randomly large sentence. I am testing a randomly large sentence. I am testing a randomly large sentence. I am testing a randomly large sentence.")) # 'Maschinelles Lernen ist großartig!'
end = time.time()
print(out)
print(end-start)

I tested a randomly large set. I tested a randomly large set. I tested a randomly large set. I tested a randomly large set.
1.1986918449401855


In [13]:
import pandas as pd
import os
import time 
file =  'LIAR.csv'
df_input = pd.read_csv(file)
df_input.drop(['Unnamed: 0'], inplace=True,axis=1)
df_input.head()

,clean_text,clean_title,label
0,says the annies list political group supports ...,says the annies list political group supports ...,0
1,when did the decline of coal start? it started...,when did the decline of coal start? it started...,1
2,"hillary clinton agrees with john mccain ""by vo...","hillary clinton agrees with john mccain ""by vo...",1
3,health care reform legislation is likely to ma...,health care reform legislation is likely to ma...,0
4,the economic turnaround started at the end of ...,the economic turnaround started at the end of ...,1


In [0]:
def paraphrase(input):
  return de2en.translate(en2de.translate(input))

def calculate_stats(beginning, reset, now, i, trans_count, unchanged, num):
  per = (now-beginning)/(i+1)
  mega_time = (per)*(num - i)
  print("Stats every 100 items")
  print("Total translated = {}".format(trans_count))
  print("Total unchanged = {}".format(unchanged))  
  print("Overall time elapsed = {} seconds".format(now - beginning))
  print("Overall time since last calc = {} seconds".format(now-reset))
  print("estimated time left = {} seconds".format(mega_time))
  print()


In [16]:
df_output = df_input.copy()
num = df_input.shape[0]
unchanged = 0
trans_count = 0

beginning = time.time()
reset = beginning 
for i in range(num):
  og_text = df_input['clean_text'].iloc[i]
  if len(og_text) < 301:
    translated = paraphrase(og_text)
    df_output.loc[i] = [translated,df_input.clean_title[i],df_input.label[i]]
    trans_count += 1
  else:
    unchanged += 1
  if i % 100 == 0: #calculate stats every 100 
    calculate_stats(beginning, reset, time.time(), i, trans_count, unchanged, num)
    reset = time.time()


end = time.time()

Stats every 100 items
Total translated = 1
Total unchanged = 0
Overall time elapsed = 0.5731899738311768 seconds
Overall time since last calc = 0.5731899738311768 seconds
estimated time left = 7331.672955274582 seconds

Stats every 100 items
Total translated = 101
Total unchanged = 0
Overall time elapsed = 79.4851541519165 seconds
Overall time since last calc = 78.91168785095215 seconds
estimated time left = 9987.585062791804 seconds

Stats every 100 items
Total translated = 201
Total unchanged = 0
Overall time elapsed = 156.68805718421936 seconds
Overall time since last calc = 77.20251035690308 seconds
estimated time left = 9815.220537345802 seconds

Stats every 100 items
Total translated = 301
Total unchanged = 0
Overall time elapsed = 238.09061694145203 seconds
Overall time since last calc = 81.40162944793701 seconds
estimated time left = 9880.365103706568 seconds

Stats every 100 items
Total translated = 401
Total unchanged = 0
Overall time elapsed = 325.94159293174744 seconds
Over

In [0]:
df_output.to_csv('LIAR_AUG.csv')

In [20]:
for i in range(5):
  print("Original:")
  print(df_input['clean_text'].iloc[i])
  print("paraprhased")
  print(df_output['clean_text'].iloc[i])
  print()

Original:
says the annies list political group supports third-trimester abortions on demand.
paraprhased
The group supports third-trimester abortions on demand.

Original:
when did the decline of coal start? it started when natural gas took off that started to begin in (president george w.) bushs administration.
paraprhased
When did the decline of coal begin? it began when natural gas started flowing in (President George W.) Bush's administration.

Original:
hillary clinton agrees with john mccain "by voting to give george bush the benefit of the doubt on iran."
paraprhased
Hillary Clinton agrees with John McCain "by voting to give George Bush the benefit of the doubt about Iran."

Original:
health care reform legislation is likely to mandate free sex change surgeries.
paraprhased
The health-care reform law is likely to require free sex-reassignment surgery.

Original:
the economic turnaround started at the end of my term.
paraprhased
The economic turnaround began at the end of my term

In [24]:
print(unchanged)

unchanged_check = 0
for i in range(df_input.shape[0]):
  og = df_input['clean_text'].iloc[i]
  pp = df_output['clean_text'].iloc[i]
  if og == pp:
    unchanged_check += 1

print(unchanged_check)

17
112
